# Building a Custom Agent with Flotorch and CrewAI

This notebook demonstrates how to build and run a simple AI agent using the Flotorch platform and the CrewAI framework. The agent will be equipped with a custom tool to perform a specific function.

### Prerequesit
Configure model, API key in Flotroch console (https://console.flotorch.cloud/)

### Viewing logs
Logs can be viewed in logs tab in Flotroch console (https://console.flotorch.cloud/)

### Key Objectives:
- Install the required Flotorch package.
- Define a custom tool for text analysis.
- Configure an agent with a specific role, goal, and tool.
- Utilize the Flotorch LLM to power the agent's reasoning capabilities.
- Assemble and execute a task with the agent.

## 1. Environment Setup and Imports

The following cells handle the complete setup of the environment. This includes:
1. **Installing** the necessary Flotorch ADK package.
2. **Importing** essential libraries.
3. **Configuring** the API credentials and model endpoint.

In [ ]:
# install flotorch adk package
%pip install --pre flotorch

In [ ]:
%pip install crewai crewai_tools crewai_tools[mcp]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
FLOTORCH_MODEL = "<flotorch model>"

In [ ]:
# Import CrewAI and related modules
import datetime
from flotorch.crewai.llm import FlotorchCrewAILLM
from crewai import Agent, Task, Crew
from crewai.tools import tool

print("Imported necessary libraries successfully")

## 2. Defining a Custom Tool

To extend the agent's capabilities, we define a custom tool. The `@tool` decorator from CrewAI transforms a standard Python function into a tool that the agent can utilize. This example creates an `analyze_text` tool, which takes a string as input and returns a statistical analysis. The function's docstring serves as its description, helping the agent understand how and when to use it.

In [ ]:
@tool
def analyze_text(text: str) -> str:
    """
    Analyze text and provide comprehensive statistics.

    Args:
        text (str): Text to analyze

    Returns:
        str: Formatted text analysis with statistics
    """
    word_count = len(text.split())
    char_count = len(text)
    char_count_no_spaces = len(text.replace(' ', ''))
    sentences = text.count('.') + text.count('!') + text.count('?')

    return f"""Text Analysis:
- Words: {word_count}
- Characters (with spaces): {char_count}
- Characters (without spaces): {char_count_no_spaces}
- Sentences: {sentences}"""


print("Custom tool defined successfully.")

## 3. Configuring the Language Model

The agent requires a Large Language Model (LLM) to act as its brain. We instantiate `FlotorchCrewAILLM`, passing our API credentials and the desired model ID. This object serves as the bridge between our CrewAI agent and the powerful reasoning capabilities of the Flotorch platform.

In [ ]:
model  =  FlotorchCrewAILLM(
        model_id=FLOTORCH_MODEL,
        api_key=FLOTORCH_API_KEY,
        base_url=FLOTORCH_BASE_URL,
    )

print(f"Flotorch LLM model configured: {model.model}")

## 4. Creating the Agent and Task

With the tool and LLM configured, we now define the agent's persona and its assigned task.

- **Agent**: We define an agent with a specific `role`, `goal`, and `backstory`. Critically, we pass the `analyze_text` tool in the `tools` list, granting the agent access to it. The `goal` is crafted to strictly guide the agent on how to use the tool and format its final answer.
- **Task**: We create a task that provides a clear `description` of what needs to be done and the `expected_output`. This guides the agent in executing its goal.

In [ ]:
agent = Agent(
    role="Text Analysis Agent",
    goal=(
        "You are a strict text analysis agent. "
        "You MUST call only the `analyze_text` tool to produce your answer. "
        "Your final message MUST be EXACTLY and ONLY the raw string returned by the tool. "
        "Do not add, remove, or reformat any characters. Do not prepend or append any text such as 'Final Answer'."
        "user_query:{query}"
    ),
    backstory=(
        "An expert analyst that never answers without running text through the `analyze_text` tool. "
        "You never post-process tool outputs. You copy the tool's output verbatim as your final answer."
    ),
    tools=[analyze_text],
    allow_delegation=False,
    verbose=False,
    llm=model,
)

task = Task(
    description=(
        "Analyze the user query  using ONLY the `analyze_text` tool. "
        "Return EXACTLY and ONLY the tool's raw output. "
        "Don't alter the tool output you just need to pass the output whatever you get from the tool"
        "user_query: {query}"
    ),
    expected_output=(
        "result from the analyze_text tool"
    ),
    agent=agent
    )
# Create custom tools and agent
print(f"Agent and Task created successfully: {agent.role}")

## 5. Assembling and Running the Crew

The final step is to assemble the components into a `Crew` and execute the task. The `Crew` object orchestrates the workflow, allowing the agent to perform its task. We call the `kickoff()` method with the user's input to start the process. The agent will then autonomously use the provided tool to generate the final response.

In [ ]:
crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=False
)

response = crew.kickoff(inputs = {"query":"Analyze this text:'it is a sample text given for the testing'"})
print(f"Assistant: {response.raw}")


## Summary

This notebook demonstrated the end-to-end process of creating a functional AI agent using the Flotorch ADK and CrewAI.

The key components included:

1. **Custom Tool Implementation**: A Python function was seamlessly converted into a usable tool for the agent.
2. **Model Configuration**: The `FlotorchCrewAILLM` was configured to provide the agent with its reasoning ability.
3. **Agent & Task Creation**: A specialized agent was defined with a clear role, goal, and task, ensuring predictable behavior.
4. **Crew Execution**: The agent and task were assembled into a crew and executed, successfully using the custom tool to produce the final output.
    
This example showcases the power and flexibility of the Flotorch platform for building customized, tool-augmented AI agents.
   
  